Churn Analysis

The notebook contains a comprehensive churn analysis for an e-commerce website. The analysis focuses on identifying patterns and reasons behind customer churn, applying machine learning models, and providing actionable insights to reduce churn. Key areas include EDA, feature engineering, model building, and evaluation metrics.

In [ ]:
# Data Handling
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Data Imputing
from sklearn.impute import KNNImputer

# Modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

: 